In [113]:
import pandas as pd
#import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import datetime
#import re
#import copy

In [11]:
vt_tax = pd.read_csv(r'D:\AS\vt_tax.csv')
vt_tax = vt_tax[vt_tax['Pur Cd']==58]
vt_tax = vt_tax[vt_tax['Tax Mode']!='B']
#vt_tax=vt_tax[vt_tax['Collected By']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)

In [13]:
vt_tax = vt_tax[vt_tax['Off Cd']==1]

In [18]:
vt_tax_clear=pd.read_csv(r'D:\Jharkhand python\vt_tax_clear.csv')
vt_tax_clear = vt_tax_clear[vt_tax_clear['Off Cd']==1]

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
vt_tax_exem = pd.read_csv(r'D:\AS\vt_tax_exem.csv')
vt_tax_exem = vt_tax_exem[vt_tax_exem['Off Cd']==1]

In [19]:
vt_owner = pd.read_csv(r'D:\AS\vt_owner.csv')
vt_owner = vt_owner[vt_owner['Off Cd']==1]

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
in_vt_tax = set(vt_tax['Regn No'].unique())
in_vt_owner = set(vt_owner['Regn No'].unique())
not_in_vt_owner = in_vt_tax-in_vt_owner
common_regn_no = in_vt_owner.intersection(in_vt_tax)
len(common_regn_no)

973724

In [23]:
#Fiters to be added are
#Regn Dt > 1st Apr 2003
#1st Apr 2017 < Tax Upto

In [26]:
vt_owner['Regn Dt'] = vt_owner.loc[:,'Regn Dt'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
vt_owner['Regn Dt'].head()

3    2014-03-20
4    2015-07-23
5    2013-01-19
10   2007-01-05
11   2013-04-25
Name: Regn Dt, dtype: datetime64[ns]

In [35]:
x = datetime.strptime('01-Apr-2003', '%d-%b-%Y')
vt_owner = vt_owner[vt_owner['Regn Dt'] > x]
min(vt_owner['Regn Dt'].to_list())

Timestamp('2003-04-02 00:00:00')

In [147]:
m = datetime.date(2017,9,1)
n = datetime.date(2017,4,1)
regn_no_list = []
table_list = []
last_paid_date_list = []
rcpt_list = []
f = 1
ff =0
for i in list(common_regn_no):
    #print(f,ff)
    f = f+1
    tax_df = vt_tax.loc[vt_tax['Regn No']==i]
    tax_clear_df = vt_tax_clear.loc[vt_tax_clear['Regn No']==i]
    tax_exem_df = vt_tax_exem.loc[vt_tax_exem['Regn No']==i]
    x = set()
    y = set()
    z = set()
    if len(tax_clear_df)>0:
        tax_clear_df['Op Dt'] = tax_clear_df.loc[:,'Op Dt'].apply(lambda x: datetime.datetime.strptime(x[:10], '%Y-%m-%d'))    
        x = set(tax__clear_df['Op Dt'].to_list())
    if len(tax_df)>0:
        tax_df['Rcpt Dt'] = tax_df.loc[:,'Rcpt Dt'].apply(lambda x: datetime.datetime.strptime(x[:11], '%d-%b-%Y'))
        y = set(tax_df['Rcpt Dt'].to_list())
    if len(tax_exem_df)>0:
        tax_exem_df['Op Dt'] = tax_clear_df.loc[:,'Op Dt'].apply(lambda x: datetime.datetime.strptime(x[:11], '%d-%b-%Y'))
        z = set(tax_exem_df['Op Dt'].to_list())

    all_dates = x.union(y).union(z)
    all_dates = list(all_dates)
    all_dates = [j for j in all_dates if str(j) != 'nan']
    last_tax_paid_on = min(all_dates)
    #print(i,type(last_tax_paid_on))
    #last_tax_paid_on
    w = str(last_tax_paid_on)
    lpo = datetime.date(int(w[:4]),int(w[5:7]),int(w[8:10]))
    #print(type(lpo),type(y),type(z))
    if lpo < n or lpo > m:
        continue
    ff=ff+1
    #print(last_tax_paid_on)
    p1 = tax_df['Rcpt No'].to_list()
    p2 = tax_df['Rcpt Dt'].to_list()
    pp = pd.DataFrame(list(zip(p1,p2)),columns=['ID','Dt'])
    pp['Table'] = 'tax'
    #print(pp)
    q1 = tax_clear_df['Appl No'].to_list()
    q2 = tax_clear_df['Op Dt'].to_list()
    qq = pd.DataFrame(list(zip(q1,q2)),columns=['ID','Dt'])
    qq['Table'] = 'clear'
    r1 = tax_exem_df['Perm No'].to_list()
    r2 = tax_exem_df['Op Dt'].to_list()
    rr = pd.DataFrame(list(zip(r1,r2)),columns=['ID','Dt'])
    rr['Table'] = 'exem'
    #r = tax_exem_df['Perm No']
    df = pd.concat([pp,qq,rr])
    #print(df)
    table = df[df['Dt']==last_tax_paid_on]['Table'].to_list()[0]
    rcpt = df[df['Dt']==last_tax_paid_on]['ID'].to_list()[0]
    #print(table) 
    print(f,ff,i,table,last_tax_paid_on,rcpt)
    regn_no_list.append(i)
    table_list.append(table)
    last_paid_date_list.append(last_tax_paid_on)
    rcpt_list.append(rcpt)
    
    
def_df = pd.DataFrame(list(zip(regn_no_list,rcpt_list,last_paid_date_list,table_list)),columns = ['Regn No','Rcpt No','Last Paid on','Table'])

C:\Users\CDMA\anaconda3\lib\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


10 1 AS01DJ7429 tax 2017-07-07 00:00:00 AS1D170700001205
25 2 AS01DH1549 tax 2017-04-12 00:00:00 AS1D170400002655
37 3 AS01DH3561 tax 2017-04-06 00:00:00 AS1D170400001440
57 4 AS01DH0104 tax 2017-04-19 00:00:00 AS1D170400003609
72 5 AS01DJ2911 tax 2017-06-17 00:00:00 AS1D170600003557
79 6 AS01DH1146 tax 2017-04-24 00:00:00 AS1R170400014177


C:\Users\CDMA\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


159 7 AS01DG7613 tax 2017-04-04 00:00:00 AS1R170400003008
182 8 AS01DH7197 tax 2017-05-18 00:00:00 AS1D170500003230
198 9 AS01DJ4828 tax 2017-06-26 00:00:00 AS1D170600005521
205 10 AS01DJ8403 tax 2017-07-13 00:00:00 AS1D170700002241
261 11 AS01DJ5800 tax 2017-06-29 00:00:00 AS1D170600006612
271 12 AS01HC9437 tax 2017-07-05 00:00:00 AS1R170700004071
353 13 AS01DJ9227 tax 2017-07-18 00:00:00 AS1D170700003074
366 14 AS01DK5558 tax 2017-08-19 00:00:00 AS1D170800003664
393 15 AS01DJ3605 tax 2017-06-20 00:00:00 AS1D170600004198
408 16 AS01DK6079 tax 2017-08-22 00:00:00 AS1D170800004240
439 17 AS01DJ4166 tax 2017-06-22 00:00:00 AS1D170600004817
448 18 AS01DJ5608 tax 2017-06-29 00:00:00 AS1D170600006423
457 19 AS01DK8196 tax 2017-09-01 00:00:00 AS1D170900000229
459 20 AS01DJ2457 tax 2017-06-14 00:00:00 AS1D170600002930
473 21 AS01DH8343 tax 2017-05-24 00:00:00 AS1D170500004804
500 22 AS01DH3647 tax 2017-05-04 00:00:00 AS1D170500000496
535 23 AS01DK6222 tax 2017-08-23 00:00:00 AS1D170800004326


4185 144 AS01DJ8281 tax 2017-07-12 00:00:00 AS1D170700002027
4189 145 AS01DH0811 tax 2017-04-21 00:00:00 AS1D170400004437
4219 146 AS01DH9139 tax 2017-05-29 00:00:00 AS1D170500005723
4262 147 AS01DK6857 tax 2017-08-26 00:00:00 AS1D170800005126
4355 148 AS01DJ1817 tax 2017-06-10 00:00:00 AS1D170600002334
4421 149 AS01DK3951 tax 2017-08-10 00:00:00 AS1D170800001970
4425 150 AS01DK8026 tax 2017-09-01 00:00:00 AS1D170900000076
4447 151 AS01HC9682 tax 2017-07-25 00:00:00 AS1R170700014142
4459 152 AS01DH7283 tax 2017-05-20 00:00:00 AS1R170500008311
4466 153 AS01DH0793 tax 2017-04-21 00:00:00 AS1D170400004476
4468 154 AS01DH4459 tax 2017-05-08 00:00:00 AS1R170500003137
4476 155 AS01HC8552 tax 2017-05-12 00:00:00 AS1R170500004412
4491 156 AS01DJ6552 tax 2017-07-03 00:00:00 AS1D170700000364
4502 157 AS01DJ3146 tax 2017-06-17 00:00:00 AS1D170600003659
4503 158 AS01DK5601 tax 2017-08-19 00:00:00 AS1D170800003697
4547 159 AS01DJ4599 tax 2017-06-24 00:00:00 AS1D170600005347
4675 160 AS01DH8760 tax 

...

167489 5758 AS01DJ7734 tax 2017-07-09 00:00:00 AS1D170700001541
167493 5759 AS01DH4101 tax 2017-05-05 00:00:00 AS1D170500000772
167525 5760 AS01ER0350 tax 2017-06-16 00:00:00 AS1R170600007027
167530 5761 AS01DG9138 tax 2017-04-13 00:00:00 AS1R170400010651
167591 5762 AS01DH7778 tax 2017-05-22 00:00:00 AS1D170500004251
167594 5763 AS01DJ4340 tax 2017-06-22 00:00:00 AS1D170600004852
167623 5764 AS01DG6968 tax 2017-04-03 00:00:00 AS1D170400000819
167646 5765 AS01DK2335 tax 2017-08-02 00:00:00 AS1D170800000282
167715 5766 AS01DJ3412 tax 2017-06-19 00:00:00 AS1D170600004059
167742 5767 AS01DL9005 tax 2017-08-30 00:00:00 AS1D170800005869
167744 5768 AS01DK1735 tax 2017-07-29 00:00:00 AS1D170700005714
167753 5769 AS01DK1567 tax 2017-07-27 00:00:00 AS1D170700004996
167765 5770 AS01DK7694 tax 2017-08-31 00:00:00 AS1D170800006032
167778 5771 AS01DJ1691 tax 2017-06-10 00:00:00 AS1D170600002222
167811 5772 AS01DG8756 tax 2017-04-11 00:00:00 AS1R170400008371
167814 5773 AS01DJ0829 tax 2017-06-07 00

171037 5887 AS01DK4070 tax 2017-08-10 00:00:00 AS1D170800002042
171055 5888 AS01DK1475 tax 2017-07-28 00:00:00 AS1D170700005441
171082 5889 AS01DH3989 tax 2017-05-05 00:00:00 AS1D170500000657
171144 5890 AS01DJ3811 tax 2017-06-21 00:00:00 AS1D170600004408
171243 5891 AS01HC8669 tax 2017-05-19 00:00:00 AS1R170500007960
171244 5892 AS01DH2318 tax 2017-04-27 00:00:00 AS1D170400005640
171247 5893 AS01DG9314 tax 2017-04-13 00:00:00 AS1D170400003035
171305 5894 AS01DH0917 tax 2017-04-20 00:00:00 AS1D170400004030
171322 5895 AS01HC9099 tax 2017-06-15 00:00:00 AS1R170600006271
171331 5896 AS01DJ8902 tax 2017-07-17 00:00:00 AS1D170700002714
171351 5897 AS01DH8050 tax 2017-05-23 00:00:00 AS1D170500004564
171396 5898 AS01DK6626 tax 2017-08-24 00:00:00 AS1D170800004745
171399 5899 AS01DH7562 tax 2017-05-20 00:00:00 AS1D170500004006
171453 5900 AS01DK3215 tax 2017-08-07 00:00:00 AS1D170800001239
171554 5901 AS01HC9504 tax 2017-07-11 00:00:00 AS1R170700007408
171622 5902 AS01DH3502 tax 2017-04-06 00

175120 6016 AS01DJ8965 tax 2017-07-17 00:00:00 AS1D170700002831
175158 6017 AS01DK1079 tax 2017-07-27 00:00:00 AS1D170700004944
175163 6018 AS01DJ5703 tax 2017-06-29 00:00:00 AS1D170600006465
175174 6019 AS01ER0523 tax 2017-07-27 00:00:00 AS1R170700015678
175201 6020 AS01DJ4997 tax 2017-06-27 00:00:00 AS1D170600005791
175264 6021 AS01DH5991 tax 2017-04-06 00:00:00 AS1D170400001426
175283 6022 AS01DJ0945 tax 2017-06-08 00:00:00 AS1D170600001461
175301 6023 AS01DK4108 tax 2017-08-10 00:00:00 AS1D170800002134
175319 6024 AS01HC9208 tax 2017-06-20 00:00:00 AS1R170600009178
175342 6025 AS01DJ2474 tax 2017-06-15 00:00:00 AS1D170600003024
175344 6026 AS01DK3491 tax 2017-08-08 00:00:00 AS1D170800001487
175382 6027 AS01DJ8850 tax 2017-07-15 00:00:00 AS1D170700002682
175390 6028 AS01DJ6846 tax 2017-07-04 00:00:00 AS1D170700000591
175525 6029 AS01DK1049 tax 2017-07-27 00:00:00 AS1D170700004903
175564 6030 AS01DH1195 tax 2017-04-24 00:00:00 AS1D170400004900
175581 6031 AS01DH0835 tax 2017-04-21 00

178494 6145 AS01DK4052 tax 2017-08-10 00:00:00 AS1D170800002080
178506 6146 AS01DK3761 tax 2017-08-09 00:00:00 AS1R170800004416
178520 6147 AS01DK2990 tax 2017-08-05 00:00:00 AS1D170800000997
178527 6148 AS01DH3010 tax 2017-04-29 00:00:00 AS1R170400017019
178540 6149 AS01DK6358 tax 2017-08-24 00:00:00 AS1D170800004523
178583 6150 AS01DK0824 tax 2017-07-26 00:00:00 AS1D170700004660
178719 6151 AS01DK3833 tax 2017-08-09 00:00:00 AS1D170800001812
178726 6152 AS01DK3601 tax 2017-08-08 00:00:00 AS1D170800001617
178762 6153 AS01HC7789 tax 2017-04-05 00:00:00 AS1R170400003842
178796 6154 AS01DJ5302 tax 2017-06-28 00:00:00 AS1D170600006111
178835 6155 AS01DH8869 tax 2017-05-27 00:00:00 AS1D170500005431
178879 6156 AS01DK1211 tax 2017-07-27 00:00:00 AS1D170700005014
178919 6157 AS01DH5221 tax 2017-05-10 00:00:00 AS1D170500001663
178959 6158 AS01HC8751 tax 2017-05-19 00:00:00 AS1R170500007646
179013 6159 AS01DK5304 tax 2017-08-18 00:00:00 AS1D170800003411
179046 6160 AS01DJ3398 tax 2017-06-19 00

KeyboardInterrupt: 

In [149]:
f

181479

In [150]:
def_df = pd.DataFrame(list(zip(regn_no_list,rcpt_list,last_paid_date_list,table_list)),columns = ['Regn No','Rcpt No','Last Paid on','Table'])

In [ ]:
pd.to_csv(def_df,'/Sample_Defaulter_List_Python.csv')

In [126]:
z = datetime.date(2017,9,1)
y = datetime.date(2017,4,1)
w = str(last_tax_paid_on)
lpo = datetime.date(int(w[:4]),int(w[5:7]),int(w[8:10]))
#int(w[5:7])
lpo > y

False

In [ ]:
if len(tax_clear_df)>0:
    #tax_clear_df['Clear Fr'] = tax_clear_df.loc[:,'Clear Fr'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    #tax_clear_df['Clear To'] = tax_clear_df.loc[:,'Clear To'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    tax__clear_df['Op Dt'] = tax_clear_df.loc[:,'Op Dt'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))
    
    x = set(tax__clear_df['Op Dt'].to_list())
if len(tax_df)>0:
    #tax_df['Tax From'] = tax_df.loc[:,'Tax From'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
    #tax_df['Tax Upto'] = tax_df.loc[:,'Tax Upto'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
    tax_df['Rcpt Dt'] = tax_df.loc[:,'Rcpt Dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%b-%Y'))
    y = set(tax_df['Rcpt Dt'].to_list())
if len(tax_exem_df)>0:
    #tax_exem_df['Exem Fr'] = tax_clear_df.loc[:,'Exem Fr'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
    #tax_exem_df['Exem To'] = tax_clear_df.loc[:,'Exem To'].apply(lambda x: datetime.strptime(x, '%d-%b-%Y'))
    tax__exem_df['Op Dt'] = tax_clear_df.loc[:,'Op Dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%b-%Y'))
    z = set(tax__exem_df['Op Dt'].to_list())
all_dates = x.union(y).union(z)
last_tax_paid_on = min(list(all_dates))
print(last_tax_paid_on)

In [74]:
vt_tax_exem.head()

,Number of Records,Exem By,Exem Fr,Exem To,Off Cd,Op Dt,Perm Dt,Perm No,Regn No,Remark,State Cd,User Cd
0,1,MVI,17-Feb-2018,16-Feb-2023,1,14-Feb-2019 14:24:58,14-Feb-2019,AMA0002,AMA0002,RC RENEWAL,AS,1702019680
1,1,DTO,21-Nov-2017,20-Nov-2022,1,30-Nov-2017 14:05:17,29-Nov-2017,AMA0188,AMA0188,R C RENEW,AS,1709030987
2,1,DTO,26-Apr-2019,19-Nov-2023,1,26-Apr-2019 10:28:06,26-Apr-2019,AMA2639,AMA2639,TAX EXAMTED,AS,1818038673
3,1,DTO,20-Nov-2018,25-Apr-2019,1,25-Apr-2019 13:49:59,20-Nov-2018,AMA2639,AMA2639,Transfer of Ownership,AS,1818038673
4,1,MVI,05-Jan-2019,11-Jul-2022,1,05-Jan-2019 14:05:35,04-Jan-2019,AMA3733,AMA3733,RC RENEWAL,AS,1702019680
